# Sessionize by IP

1. Clean and create temporary folder.  Unzip data.

In [3]:
import shutil
import os
import gzip

scriptpath = os.getcwd()
tempdir = '.tmp'

if os.path.exists(tempdir):
    shutil.rmtree(tempdir)
os.makedirs(tempdir)

extractedlog = 'web.log'
sourcepath = os.path.join(scriptpath, '../data/2015_07_22_mktplace_shop_web_log_sample.log.gz')
destpath = os.path.join(scriptpath, tempdir, extractedlog)
with gzip.open(sourcepath) as src, open(destpath, 'w') as dest:
    content = src.read()
    dest.write(content)